# 4. Dynamic Programming

## 4.1. Intro

policy iteration, value iteration

policy evaluation, policy improvement

## 4.2. Cliff Walking: 悬崖漫步

In [11]:
import copy

class CliffWalkingEnv:
  """悬崖环境"""
  def __init__(self, ncol=12, nrow=4):
    self.ncol = ncol # 网格世界的列
    self.nrow = nrow # 网络世界的行
    # 转移矩阵P[state][action] = [(p, next_state, reward, done)]
    # 包含下一个状态和奖励
    self.P = self.createP()
    
  def createP(self):
    # 初始化
    P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]
    # 4 个动作，change[0,1,2,3]: 上下左右。原点(0, 0)
    # 定义左上角
    change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
    for i in range(self.nrow):
      for j in range(self.ncol):
        for a in range(4):
          # 位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
          if i == self.nrow - 1 and j > 0:
            P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0, True)]
            continue
          # 其他位置
          next_x = min(self.ncol - 1, max(0, j + change[a][0]))
          next_y = min(self.nrow - 1, max(0, i + change[a][1]))
          next_state = next_y * self.ncol + next_x
          reward = -1
          done = False
          # 下一个位置在悬崖或者终点
          if next_y == self.nrow - 1 and next_x > 0:
            done = True
            if next_x != self.ncol - 1: # 下一个位置在悬崖
              reward = -100
          P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
    return P

## 4.3. Policy Iteration: 策略迭代

### 4.3.1. Policy Evaluation：策略评估

$$
V^\pi(s) = \sum_{a \in A}\pi(a|s)\left(r(s, a) + \gamma\sum_{s' \in S}p(s'|s, a) V^\pi(s')\right)
$$

$$
V^{k+1}(s) = \sum_{a \in A}\pi(a|s)\left(r(s, a) + \gamma\sum_{s' \in S}P(s'|s, a) V^k(s')\right)
$$

### 4.3.2. Policy Improvement: 策略提升

测录提升定理：

假设存在确定性策略$\pi'$, 在任意状态$s$, 有$Q^\pi(s, \pi'(s)) \geq V^\pi(s)$, 则有$V^{\pi'}(s) \geq V^\pi(s)$

贪心：

$$
\pi'(s) = \argmax_a Q^\pi(s, a) = \argmax_a \left\{r(s, a) + \gamma\sum_{s'}P(s'|s, a)V^\pi(s')\right\}
$$

### 4.3.3. Policy Iteration Algorithm
$$
\pi^0 \stackrel{策略评估}{\rightarrow} V^{\pi^0} \stackrel{策略提升}\rightarrow ... \pi^i \stackrel{策略评估}{\rightarrow} V^{\pi^i} \stackrel{策略提升}\rightarrow ... \stackrel{策略提升}\rightarrow \pi^*
$$

伪代码：
1. 随机初始化策略$\pi(s)$和价值函数$V(s)$, 
2. **while** $\Delta > \theta$ **do**: (策略评估循环)
   1. $\Delta \leftarrow 0$
   2. $\forall s \in \mathcal{S}$
      1. $v \leftarrow V(S)$
      2.  $V(s) \leftarrow r(s, \pi(s)) + \gamma\sum_{s'}P(s' | s, \pi(s))V(s')$
      3.  $\Delta \leftarrow \max(\Delta, |v - V(s)|)$
3. **end while**
4. $\pi_{old} \leftarrow \pi$
5. 对每个状态$s \in \mathcal{S}$:
   1. $\pi(s) \leftarrow \argmax_{a} r(s, a) + \gamma\sum_{s'}P(s'|s, a)V(s')$
6. 若$\pi_{old} = \pi$, 则停止返回$V, \pi$, 否则转到策略评估循环

In [12]:
class PolicyIteration:
  """ 策略迭代算法 """
  def __init__(self, env, theta, gamma):
    self.env = env
    self.v = [0] * self.env.ncol * self.env.nrow
    self.pi = [[0.25, 0.25, 0.25, 0.25] for i in range(self.env.ncol * self.env.nrow)]
    self.theta = theta 
    self.gamma = gamma
    
  def policy_evaluation(self): 
    cnt = 1
    while 1:
      max_diff = 0
      new_v = [0] * self.env.ncol * self.env.nrow
      for s in range(self.env.ncol * self.env.nrow):
        qsa_list = [] # 开始计算s状态下所有的Q(s, a)价值
        for a in range(4):
          qsa = 0
          for res in self.env.P[s][a]:
            p, next_state, r, done = res
            qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
          qsa_list.append(qsa)
        new_v[s] = max(qsa_list)
        max_diff = max(max_diff, abs(new_v[s] - self.v[s]))
      self.v = new_v
      if max_diff < self.theta: break 
      cnt += 1
    print("策略评估进行%d轮后完成" % cnt)

    
  def policy_improvement(self): # 根据价值函数导出贪婪策略
    for s in range(self.env.nrow * self.env.ncol):
      qsa_list = []
      for a in range(4):
        qsa = 0
        for res in self.env.P[s][a]:
          p, next_state, r, done = res
          qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
        qsa_list.append(qsa)
      maxq = max(qsa_list)
      cntq = qsa_list.count(maxq) # 计算几个动作得到最大Q值
      # 均分这些动作的均分概率
      self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]
    print("策略提升完成")
    return self.pi

  def policy_iteration(self):
    while 1:
      self.policy_evaluation()
      old_pi = copy.deepcopy(self.pi)
      new_pi = self.policy_improvement()
      if old_pi == new_pi: break

In [13]:
def print_agent(agent, action_meaning, disaster=[], end=[]):
  print("状态价值： ")
  for i in range(agent.env.nrow):
    for j in range(agent.env.ncol):
      print('%6.6s' % ('%.3f' % agent.v[i * agent.env.ncol + j]),
            end=' ')
    print()
  
  print("策略：")
  for i in range(agent.env.nrow):
    for j in range(agent.env.ncol):
      if (i * agent.env.ncol + j) in disaster:
        print("****", end = " ")
      elif (i * agent.env.ncol + j) in end:
        print("EEEE", end = ' ')
      else:
        a = agent.pi[i * agent.env.ncol + j]
        pi_str = ''
        for k in range(len(action_meaning)):
          pi_str += action_meaning[k] if a[k] > 0 else 'o'
        print(pi_str, end=' ')
        
    print()

env = CliffWalkingEnv()
action_meaning =['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, list(range(37, 47)), [47])
  


策略评估进行15轮后完成
策略提升完成
策略评估进行1轮后完成
策略提升完成
状态价值： 
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


## 4.4. Value Iteration: 价值迭代算法

$$
V^*(s) = \max_{a\in\mathcal{A}}\left\{r(s, a) + \gamma\sum_{s' \in S}p(s'|s, a) V^*(s')\right\}
$$

$$
V^{k+1}(s) = \max_{a\in\mathcal{A}}\left\{r(s, a) + \gamma\sum_{s' \in \mathcal{S}}P(s'|s, a) V^k(s')\right\}
$$

$$
\pi(s) = \argmax_a \left\{r(s, a) + \gamma\sum_{s'}p(s'|s, a)V^{k + 1}(s')\right\}
$$

伪代码：
1. 随机初始化价值函数$V(s)$, 
2. **while** $\Delta > \theta$ **do**: (策略评估循环)
   1. $\Delta \leftarrow 0$
   2. $\forall s \in \mathcal{S}$
      1. $v \leftarrow V(S)$
      2.  $V(s) \leftarrow \max_a r(s, a) + \gamma\sum_{s'}P(s' | s, \pi(s))V(s')$
      3.  $\Delta \leftarrow \max(\Delta, |v - V(s)|)$
3. **end while**
4. 返回一个确定性策略
5. 对每个状态$s \in \mathcal{S}$:
$\pi(s) \leftarrow \argmax_{a} \left\{r(s, a) + \gamma\sum_{s'}P(s'|s, a)V(s')\right\}$

In [16]:
class ValueIteration:
  """ 价值迭代算法 """
  def __init__(self, env, theta, gamma):
    self.env = env
    self.v = [0] * self.env.ncol * self.env.nrow
    # self.pi = [[0.25, 0.25, 0.25, 0.25] for i in range(self.env.ncol * self.env.nrow)]
    self.theta = theta 
    self.gamma = gamma
    # 价值迭代结束后得到的策略
    self.pi = [None for i in range(self.env.ncol * self.env.nrow)]
    
  def value_iteration(self): 
    cnt = 0
    while 1:
      max_diff = 0
      new_v = [0] * self.env.ncol * self.env.nrow
      for s in range(self.env.ncol * self.env.nrow):
        qsa_list = [] # 开始计算s状态下所有的Q(s, a)价值
        for a in range(4):
          qsa = 0
          for res in self.env.P[s][a]:
            p, next_state, r, done = res
            qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
          qsa_list.append(qsa)
        new_v[s] = max(qsa_list)
        max_diff = max(max_diff, abs(new_v[s] - self.v[s]))
      self.v = new_v
      if max_diff < self.theta: break 
      cnt += 1
    print("价值评估进行%d轮后完成" % cnt)
    self.get_policy()

    
  def get_policy(self): # 根据价值函数导出贪婪策略
    for s in range(self.env.nrow * self.env.ncol):
      qsa_list = []
      for a in range(4):
        qsa = 0
        for res in self.env.P[s][a]:
          p, next_state, r, done = res
          qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
        qsa_list.append(qsa)
      maxq = max(qsa_list)
      cntq = qsa_list.count(maxq) # 计算几个动作得到最大Q值
      # 均分这些动作的均分概率
      self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]
    # print("策略提升完成")
    # return self.pi

env = CliffWalkingEnv()
action_meaning =['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
print_agent(agent, action_meaning, list(range(37, 47)), [47])


价值评估进行14轮后完成
状态价值： 
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


## 4.5. Frozen Lake

In [ ]:
import gym
import numpy as np

env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="ansi")  # 创建环境
env.reset()
print(env.render())  # 环境渲染,通常是弹窗显示或打印出可视化的环境

P = env.unwrapped.P

holes = set()
ends = set()
for s in P:
  for a in P[s]:
    for transition in env.P[s][a]:
      prob, next_state, reward, done = transition
      if reward == 1.0:  # 获得奖励为1,代表是目标
          ends.add(next_state)
      if done:
          holes.add(next_state)

holes = holes - ends
print("冰洞的索引:", holes)
print("目标的索引:", ends)

for a in P[14]:  # 查看目标左边一格的状态转移信息
  print(P[14][a])


SFFF
FHFH
FFFH
HFFG

冰洞的索引: {11, 12, 5, 7}
目标的索引: {15}
[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


In [21]:
action_meaning =['<', 'v', '>', '^']
theta = 1e-5
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

策略评估进行61轮后完成
策略提升完成
策略评估进行1轮后完成
策略提升完成
状态价值： 
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


In [22]:
action_meaning =['<', 'v', '>', '^']
theta = 1e-5
gamma = 0.9
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
print_agent(agent, action_meaning, [5, 7, 11, 12], [15])

价值评估进行60轮后完成
状态价值： 
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 
